In [ ]:
import torch
import numpy as np
import math
from itertools import groupby
import h5py
import numpy as np
import unicodedata
import cv2
import torch
from torch import nn
from torchvision.models import resnet50, resnet101
from torch.autograd import Variable
import torchvision
from torch.utils.data import Dataset
import string

In [ ]:
import keras
import numpy as np
import tensorflow as tf
import sys
sys.path.append('../')
from utils.lib import *

In [ ]:
data, label_file = load_datargb('',['../src/Method 2_dataset/thumbnail_augment/'])

data = np.expand_dims(data,axis=0)

data = data.reshape(-1,30,375, 3)

# data = data/255.

label = read_final_label('../src/Method 2_dataset/final_augment_handwritten.txt')

In [ ]:
# data, label = shuffle(data,label, random_state=42)

label = [i.replace('||',' ') for i in label]
label = [i.replace('|',' ') for i in label]
label = [i[1:] if i[0]==' ' else i for i in label]
label = [i[:-1] if i[-1]==' ' else i for i in label]
label = [i.replace('  ',' ') for i in label]
label = [i.replace('\n','') for i in label]

train_data = data[:4000]
train_label = label[:4000]
test_data = data[4000:]
test_label = label[4000:]

In [ ]:
import transformers
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
# model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

def ocr_image(src_img):
  pixel_values = processor(images=src_img, return_tensors="pt").pixel_values
  generated_ids = model.generate(pixel_values)
  return processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

In [ ]:
plt.imshow(data[1])

In [ ]:
label[1]

In [ ]:
import pandas as pd

d = {'file_name': label_file, 'text': label}
df = pd.DataFrame(data=d)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)
train_df, valid_df = train_test_split(train_df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)


In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [ ]:
train_dataset = IAMDataset(root_dir='../src/Method 2_dataset/thumbnail_augment/',
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='../src/Method 2_dataset/thumbnail_augment/',
                           df=test_df,
                           processor=processor)
valid_dataset = IAMDataset(root_dir='../src/Method 2_dataset/thumbnail_augment/',
                           df=valid_df,
                           processor=processor)

In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

In [ ]:
encoding = train_dataset[0]
for k,v in encoding.items():
  print(k, v.shape)

In [ ]:
image = Image.open(train_dataset.root_dir + train_df['file_name'][0]).convert("RGB")
image

In [ ]:
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

In [ ]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("results_hcr_vit/checkpoint-1500")

In [ ]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=False, 
    output_dir="results_hcr_vit/",
    logging_steps=2,
    save_steps=100,
    eval_steps=200
)

In [ ]:
from datasets import load_metric

cer_metric = load_metric("cer")

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator
)
trainer.train()

## Testing

In [ ]:
test_df

In [ ]:
import fastwer

In [ ]:
predicted_output = list()

In [ ]:
predicted_output = list()

for item in range(len(test_df)):
    img = Image.open(train_dataset.root_dir + test_df['file_name'][item]).convert("RGB")
    pixel_values = processor(images=img, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    predicted_output.append(processor.batch_decode(generated_ids, skip_special_tokens=True)[0])

In [ ]:
predicted_output

In [ ]:
a = list(test_df['text'])

In [ ]:
import fastwer

In [ ]:
fastwer.score(a, predicted_output, char_level=True)


In [ ]:
fastwer.score(a, predicted_output)


In [ ]:
def ocr_image(src_img):
  pixel_values = processor(images=src_img, return_tensors="pt").pixel_values
  generated_ids = model.generate(pixel_values)
  return processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
%load_ext tensorboard

%tensorboard --logdir results_hcr_vit/runs